In [1]:
import tensorflow as tf
# Suppress the annoying tensorflow 1.x deprecation warnings; these make console output
# impossible to parse.
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
# initialize logger to view logging output from segmentation tasks
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


In [3]:
from google.protobuf import text_format
from ffn.inference import inference
from ffn.inference import inference_pb2

DEBUG:matplotlib:CACHEDIR=/Users/jpgard/.matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /Users/jpgard/.matplotlib/fontlist-v300.json


In [4]:
config = '''image {
  hdf5: "data/test/507727402/507727402_raw.h5:raw"
}
image_mean: 78
image_stddev: 20
checkpoint_interval: 1800
seed_policy: "PolicyPeaks"
model_checkpoint_path: "training-logs/lr0.001/model.ckpt-1000000"
model_name: "fftracer.training.models.model.FFNTracerModel"
model_args: "{\\"depth\\": 9, \\"fov_size\\": [49, 49, 1], \\"deltas\\": [8, 8, 0]}"
segmentation_output_dir: "results/mozak"
inference_options {
  init_activation: 0.95
  pad_value: 0.05
  move_threshold: 0.9
  min_boundary_dist { x: 1 y: 1 z: 0}
  segment_threshold: 0.6
  min_segment_size: 1000
}'''
req = inference_pb2.InferenceRequest()
_ = text_format.Parse(config, req)

In [ ]:
runner = inference.Runner()
runner.start(req)
canvas, alignment = runner.make_canvas((0, 0, 0), (1, 9429, 7601)) # sizes in (z, y, x)

DEBUG:root:Received request:
image_mean: 78.0
image_stddev: 20.0
model_name: "fftracer.training.models.model.FFNTracerModel"
model_args: "{\"depth\": 9, \"fov_size\": [1, 49, 49], \"deltas\": [8, 8, 0]}"
model_checkpoint_path: "training-logs/lr0.001/model.ckpt-1000000"
inference_options {
  init_activation: 0.949999988079071
  pad_value: 0.05000000074505806
  move_threshold: 0.8999999761581421
  min_boundary_dist {
    x: 1
    y: 1
    z: 0
  }
  segment_threshold: 0.6000000238418579
  min_segment_size: 1000
}
segmentation_output_dir: "results/mozak"
checkpoint_interval: 1800
seed_policy: "PolicyPeaks"
image {
  hdf5: "data/test/507727402/507727402_raw.h5:raw"
}

INFO:root:Available TF devices: [_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 757763813882681302), _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 6549946503828001155)]
INFO:root:Importing symbol FFNTracerModel from ffn.training.models.fftracer.train

In [ ]:
### EXPERIMENTAL: show the input image data to verify it is valid
import skimage
from matplotlib import pyplot as plt
skimage.io.imshow(canvas.image[0,:,:])
plt.show()
### END

DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG:matplotlib.colorbar:locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x147f5ac18>
DEBUG:matplotlib.colorbar:Using auto colorbar locator on colorbar
DEBUG:matplotlib.colorbar:locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x147f5ac18>
DEBUG:matplotlib.colorbar:Setting pcolormesh
DEBUG:matplotlib.font_manager:findfont: Matching :family=sans-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0 to DejaVu Sans ('/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf') with score of 0.050000.
DEBUG:matplotlib.axes._base:update_title_pos
DEBUG:matplotlib.axes._base:update_title_pos


<Figure size 640x480 with 2 Axes>

DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [ ]:
# Create a single segment, starting from the specified origin point.
canvas.segment_at(
    (0, 4521, 3817),  # seed starting point in (z, y, x)
    dynamic_image=inference.DynamicImage(),
    vis_update_every=1,
    vis_fixed_z=True
)

[JG] segmenting at start_pos (0, 4521, 3817)
[JG] NO self.movement_policy
> /Users/jpgard/Documents/github/ffn/ffn/inference/inference.py(523)segment_at()
    522       import ipdb;ipdb.set_trace()
--> 523       assert len(self.movement_policy) > 0
    524 

ipdb> c


DEBUG:root:.. too close to border: (0, 4521, 3817)


[JG] movement_policy: <ffn.inference.movement.FaceMaxMovementPolicy object at 0x115de2b00>
> /Users/jpgard/Documents/github/ffn/ffn/inference/inference.py(356)is_valid_pos()
    355       import ipdb;ipdb.set_trace()
--> 356       return False
    357 

ipdb> np.any(low < 0)
True
ipdb> low
array([ -24, 4497, 3817])
ipdb> self.margin
array([24, 24,  0])


In [ ]:
# read the results
import numpy as np
seg_prob = np.load("results/mozak/0/0/seg-0_0_0.prob")['qprob']
print("shape: {}".format(seg_prob.shape))
seg_prob